## Crassification Titanic
　　　From SIGNATE　【練習問題】タイタニックの生存予測

In [ ]:
# 基本ライブラリの読み込み
import numpy as np
import pandas as pd

### trainデータ

In [ ]:
# train.tsvの読み込み
train_df = pd.read_csv("train.tsv", sep="\t")
train_df.head()

In [ ]:
# インデックスとy（目的変数）を説明変数から削除して、特徴量のみをtrainXに代入する
trainX = train_df.iloc[:, 2:]
trainX.head()

In [ ]:
# 行列の桁数確認
trainX.shape

In [ ]:
# 基礎統計量の確認
trainX.describe()

#### 統計量からわかること
- numericな特徴量は５つのみ (客室のクラス（1,2,3の順に高級クラス）、年齢、乗船していた兄弟配偶者の数、乗船していた両親子供の数、運賃)
- 特徴量の中では pclass (客室のクラス)、parchの(乗船していた両親子供の数)バラつきが比較的小さい
- 特徴量の中では fare (運賃)のバラつきが比較的大きい

In [ ]:
# データ型の確認
trainX.info()

整数型、小数点型、文字列などが混在している

In [ ]:
# 欠損値の確認
trainX.isnull().sum()

"age","embarked"に欠損値を含む

In [ ]:
# trainデータの目的変数をyに代入する
y = train_df.iloc[:,[1]]
display(y.head())

In [ ]:
# 性別カラムの種別個数を確認
trainX["sex"].value_counts()

In [ ]:
# 乗船地カラムの種別個数を確認
trainX["embarked"].value_counts()

In [ ]:
# オブジェクト型の特徴量を数値型に変換
class_mapping_sex = {"male":0 ,"female":1}
class_mapping_emb = {"S": 0, "C": 1, "Q": 2} 

trainX = trainX.copy()

# loc関数で'該当'列のみを選択して、変数class_mapping_sex,embをmap関数で適用する
trainX.loc[:,"sex"] =trainX["sex"].map(class_mapping_sex)
trainX.loc[:,"embarked"] = trainX["embarked"].map(class_mapping_emb)

display(trainX.head())

In [ ]:
# trainXの欠損値のあるカラム'age'の平均値を求め、変数avg1に代入
avg1 = trainX["age"].mean()
avg1

In [ ]:
# 変数avgをnullであるカラムに穴埋めする
trainX.loc[:,"age"] = trainX["age"].fillna(avg1)
trainX.head()

In [ ]:
# 欠損値の確認
trainX.isnull().sum()

In [ ]:
# imputerを使用してembarkedの欠損値を最頻値で置き換える
from sklearn.preprocessing import Imputer

# インピュータークラスの実体化
# 欠損値'NaN'を最頻値('most_frequent')で置き換える,処理は列方向(axis=0)で行う.
imp = Imputer(missing_values='NaN',
               strategy='most_frequent',
               axis=0)

# 各特徴量の平均値を学習
imp.fit(trainX)

# 学習済みのImputerを適用し, 欠損値を置き換える.
mfq_columns = trainX.columns.values
trainX2 = pd.DataFrame(imp.transform(trainX),
                     columns=mfq_columns)

In [ ]:
# 結果表示
trainX2.head()

In [ ]:
# 欠損値の確認
trainX2.isnull().sum()

==========================================================================================================================

### testデータ

In [ ]:
# test.tsvの読み込み
test_df = pd.read_csv("test.tsv", sep="\t")
test_df.head()

In [ ]:
# インデックスを説明変数から削除して、特徴量のみをtestXに代入する
testX = test_df.iloc[:, 1:]
testX.head()

In [ ]:
# 行列の桁数確認
testX.shape

In [ ]:
# 基礎統計量の確認
testX.describe()

#### 統計量からわかること
- numericな特徴量は５つのみ (客室のクラス（1,2,3の順に高級クラス）、年齢、乗船していた兄弟配偶者の数、乗船していた両親子供の数、運賃)
- 特徴量の中では pclass (客室のクラス)、parchの(乗船していた両親子供の数)バラつきが比較的小さい
- 特徴量の中では fare (運賃)のバラつきが比較的大きい

In [ ]:
# データ型の確認
testX.info()

整数型、小数点型、文字列などが混在している

In [ ]:
# 欠損値の確認
testX.isnull().sum()

"age"に欠損値があることを確認。

In [ ]:
# オブジェクト型の特徴量を数値型に変換
# class_mapping_sex = {"male":0 ,"female":1}
# class_mapping_emb = {"S": 0, "C": 1, "Q": 2} 

testX = testX.copy()

# loc関数で該当列のみを選択して、変数class_mapping_sex,embをmap関数で適用する
testX.loc[:,"sex"] =testX["sex"].map(class_mapping_sex)
testX.loc[:,"embarked"] = testX["embarked"].map(class_mapping_emb)

display(testX.head())

In [ ]:
# trainXの欠損値のあるカラム'age'の平均値を求め、変数avg2に代入
avg2 = testX["age"].mean()
avg2

In [ ]:
# 変数avg2をnullであるカラムに穴埋めする
testX.loc[:,"age"] = testX["age"].fillna(avg2)
testX.head()

In [ ]:
# 欠損値の確認
testX.isnull().sum()

==========================================================================================================================

### trainデータの交差検証

In [ ]:
# 交叉検証　Holdout法
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(trainX2,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=1)

In [ ]:
# 整形
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
# ライブラリからアルゴリズムの読み込み
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# set pipelines for different algorithms
pipe_LR = Pipeline([('scl',StandardScaler()),
                       ('est',LogisticRegression(random_state=1))])
pipe_RF = Pipeline([('scl',StandardScaler()),
                        ('est',RandomForestClassifier(random_state=1))])
pipe_GB = Pipeline([('scl',StandardScaler()),
                        ('est',GradientBoostingClassifier(random_state=1))])
pipe_PCARF = Pipeline([('scl',StandardScaler()),
                          ('pca',PCA(random_state=1)),
                          ('est',RandomForestClassifier(random_state=1))])
pipe_PCAGB = Pipeline([('scl',StandardScaler()),
                          ('pca',PCA(random_state=1)),
                          ('est',GradientBoostingClassifier(random_state=1))])
pipe_SVC = Pipeline([('scl',StandardScaler()),
                          ('est',SVC(random_state=1))])
pipe_LSVC = Pipeline([('scl',StandardScaler()),
                          ('est',LinearSVC(random_state=1))])
pipe_KNC = Pipeline([('scl',StandardScaler()),
                          ('est',KNeighborsClassifier())])
pipe_GNB = Pipeline([('scl',StandardScaler()),
                          ('est',GaussianNB())])
pipe_PER = Pipeline([('scl',StandardScaler()),
                          ('est',Perceptron(random_state=1))])
pipe_SGD = Pipeline([('scl',StandardScaler()),
                          ('est',SGDClassifier(random_state=1))])
pipe_DTC = Pipeline([('scl',StandardScaler()),
                          ('est',DecisionTreeClassifier(random_state=1))])

In [ ]:
# Pipelineの学習
pipe_LR.fit(X_train, y_train)
pipe_RF.fit(X_train, y_train)
pipe_GB.fit(X_train, y_train)
pipe_PCARF.fit(X_train, y_train)
pipe_PCAGB.fit(X_train, y_train)
pipe_SVC.fit(X_train, y_train)
pipe_LSVC.fit(X_train, y_train)
pipe_KNC.fit(X_train, y_train)
pipe_GNB.fit(X_train, y_train)
pipe_PER.fit(X_train, y_train)
pipe_SGD.fit(X_train, y_train)
pipe_DTC.fit(X_train, y_train)

In [32]:
# 評価（roc_aucスコアとする）
from sklearn.metrics import roc_auc_score
print('-'*40)
print('pipe_LR_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_LR.predict(X_train)))
print('pipe_LR_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_LR.predict(X_test)))
print('-'*40)
print('pipe_RF_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_RF.predict(X_train)))
print('pipe_RF_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_RF.predict(X_test)))
print('-'*40)
print('pipe_GB_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_GB.predict(X_train)))
print('pipe_GB_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_GB.predict(X_test)))
print('-'*40)
print('pipe_PCARF_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_PCARF.predict(X_train)))
print('pipe_PCARF_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_PCARF.predict(X_test)))
print('-'*40)
print('pipe_PCAGB_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_PCAGB.predict(X_train)))
print('pipe_PCAGB_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_PCAGB.predict(X_test)))
print('-'*40)
print('pipe_SVC_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_SVC.predict(X_train)))
print('pipe_SVC_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_SVC.predict(X_test)))
print('-'*40)
print('pipe_LSVC_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_LSVC.predict(X_train)))
print('pipe_LSVC_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_LSVC.predict(X_test)))
print('-'*40)
print('pipe_KNC_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_KNC.predict(X_train)))
print('pipe_KNC_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_KNC.predict(X_test)))
print('-'*40)
print('pipe_GNB_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_GNB.predict(X_train)))
print('pipe_GNB_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_GNB.predict(X_test)))
print('-'*40)
print('pipe_PER_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_PER.predict(X_train)))
print('pipe_PER_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_PER.predict(X_test)))
print('-'*40)
print('pipe_SGD_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_SGD.predict(X_train)))
print('pipe_SGD_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_SGD.predict(X_test)))
print('-'*40)
print('pipe_DTC_Train:%.3f'% roc_auc_score(y_train,
                                          pipe_DTC.predict(X_train)))
print('pipe_DTC_Test:%.3f' % roc_auc_score(y_test,
                                          pipe_DTC.predict(X_test)))
print('-'*40)

----------------------------------------
pipe_LR_Train:0.793
pipe_LR_Test:0.763
----------------------------------------
pipe_RF_Train:0.977
pipe_RF_Test:0.705
----------------------------------------
pipe_GB_Train:0.922
pipe_GB_Test:0.747
----------------------------------------
pipe_PCARF_Train:0.974
pipe_PCARF_Test:0.735
----------------------------------------
pipe_PCAGB_Train:0.974
pipe_PCAGB_Test:0.705
----------------------------------------
pipe_SVC_Train:0.841
pipe_SVC_Test:0.791
----------------------------------------
pipe_LSVC_Train:0.796
pipe_LSVC_Test:0.773
----------------------------------------
pipe_KNC_Train:0.872
pipe_KNC_Test:0.739
----------------------------------------
pipe_GNB_Train:0.800
pipe_GNB_Test:0.733
----------------------------------------
pipe_PER_Train:0.671
pipe_PER_Test:0.685
----------------------------------------
pipe_SGD_Train:0.684
pipe_SGD_Test:0.625
----------------------------------------
pipe_DTC_Train:0.992
pipe_DTC_Test:0.689
------------

pipe_SVCを採用する

==========================================================================================================================

### パラメータチューニング

In [ ]:
# パラメータグリッドの設定
from sklearn.model_selection import GridSearchCV

param_grid_SVC=[{
                 'est__kernel':['linear','rbf','poly','sigmoid'],
                 'est__C':[1.0,0.1,0.01]
                }] 

In [ ]:
print('探索空間:%s' %param_grid_SVC)
gs=GridSearchCV(estimator=pipe_SVC,
                param_grid=param_grid_SVC,
                scoring='accuracy',
                cv=3,
                n_jobs=-1)

In [35]:
gs=gs.fit(trainX2,y.values.ravel())

# 探索した結果のベストスコアとパラメータの取得
print('Best Score:', gs.best_score_)
print('Best Params', gs.best_params_)

Best Score: 0.8134831460674158
Best Params {'est__C': 1.0, 'est__kernel': 'rbf'}


In [ ]:
# 変数predに予測値を代入
pred = pipe_SVC.predict(testX)
pred

==========================================================================================================================

### 提出用CSVの作成

In [ ]:
# 提出用CSVの読み込み
sample = pd.read_csv("sample_submit.tsv", sep="\t", header=None)
sample.head()

In [ ]:
# 提出用csvの正解カラム'1'にpredを代入
sample[1] = pred
sample.head()

In [ ]:
sample.to_csv("submit6.csv",index=None,header=None)